<center><h1>Treasure Hunt In The Enchanted Forest</h1></center>

# Description
This Python script imports necessary libraries for creating a graphical user interface (GUI) for playing a treasure hunt game in an enchanted forest. It imports tkinter for GUI components, tkinter.font for font handling, PIL for image processing, heapq for priority queue implementation, and random for generating random values.

In [1]:
import tkinter as tk
from tkinter import font
from PIL import Image, ImageTk
import heapq
import random

# Block Class

 # Description
The `Block` class represents a block in the grid of the treasure hunt game. Each block has a specific type, such as obstacle, animal, or free.#

## Attributes
- `block_type`: A string representing the type of the block, e.g., 'obstacle', 'animal', or 'free#'.

## Methods
- `__init__(self, block_type)`: Initializes a block object with the specified block class!


In [2]:
class Block:
    def __init__(self, block_type):
        self.block_type = block_type

# Player Class

## Description
The `Player` class represents a player in the treasure hunt game. The player navigates the grid to find the treasure using various search algorithms, such as A* (A-star) search and UCS (Uniform Cost Search).

## Attributes
- `grid`: The grid object representing the game environment.
- `position`: The current position of the player.
- `end_pos`: The position of the treasure (end goal).
- `total_cost`: The total cost incurred while searching for the treasure.
- `path_taken`: The path taken by the player to reach the treasure.
- `search_algorithm`: The search algorithm used by the player ('A*' or 'UCS').

## Methods
- `__init__(self, grid, start_pos, end_pos, search_algorithm)`: Initializes a player object with the specified grid, start position, end position, and search algorithm.
- `move_to_end(self)`: Moves the player to the end position using the selected search algorithm.
- `a_star_search(self)`: Performs the A* search algorithm to find the shortest path to the end position.
- `uniform_cost_search(self)`: Performs the Uniform Cost Search algorithm to find the shortest path to the end position.
- `print_path_details(self, path_num, path, cost)`: Prints details of the path taken by the player, including the path number, path coordinates, and total cost.
- `calculate_cost(self, block_type)`: Calculates the cost of moving to a neighboring blocksure hunt adventure!


In [3]:
class Player:
    def __init__(self, grid, start_pos, end_pos, search_algorithm):
        self.grid = grid
        self.position = start_pos
        self.end_pos = end_pos
        self.total_cost = 0
        self.path_taken = []
        self.search_algorithm = search_algorithm

    def move_to_end(self):
        if self.search_algorithm == "A*":
            self.a_star_search()
        elif self.search_algorithm == "UCS":
            self.uniform_cost_search()
        else:
            print("Invalid search algorithm selected.") # This will never run technically because we have not created a button for it.
    def a_star_search(self):
        visited = set()
        heap = [(0, self.position, [], 0)]  # Priority queue of (f-value, position, path, total_cost)
        path_num = 0
        while heap:
            f_value, position, path, total_cost = heapq.heappop(heap)
            if position == self.end_pos:
                path_num += 1
                self.path_taken = path + [position]
                self.print_path_details(path_num, path + [position], total_cost)
                self.total_cost = total_cost
                break
            if position in visited:
                continue
            visited.add(position)
            y, x = position
            neighbors = [(y - 1, x), (y + 1, x), (y, x - 1), (y, x + 1)]
            for neighbor in neighbors:
                if 0 <= neighbor[0] < self.grid.height and 0 <= neighbor[1] < self.grid.width:
                    neighbor_block = self.grid.grid[neighbor[0]][neighbor[1]].block_type
                    if neighbor_block != 'obstacle':
                        g_cost = self.calculate_cost(neighbor_block)  # Cost to move to neighbor
                        h_cost = abs(neighbor[0] - self.end_pos[0]) + abs(neighbor[1] - self.end_pos[1])  # Manhattan distance heuristic
                        f_value = g_cost + h_cost
                        heapq.heappush(heap, (f_value, neighbor, path + [position], total_cost + g_cost))
        if path_num == 0:
            print("No path found to the end position!")

    def uniform_cost_search(self):
        visited = set()
        heap = [(0, self.position, [])]  # Priority queue of (cost, position, path)
        path_num = 0
        while heap:
            cost, position, path = heapq.heappop(heap)
            if position == self.end_pos:
                path_num += 1
                self.path_taken = path + [position]
                self.print_path_details(path_num, path + [position], cost)
                self.total_cost = cost
                break
            if position in visited:
                continue
            visited.add(position)
            y, x = position
            neighbors = [(y - 1, x), (y + 1, x), (y, x - 1), (y, x + 1)]
            for neighbor in neighbors:
                if 0 <= neighbor[0] < self.grid.height and 0 <= neighbor[1] < self.grid.width:
                    neighbor_block = self.grid.grid[neighbor[0]][neighbor[1]].block_type
                    neighbor_cost = self.calculate_cost(neighbor_block)
                    if neighbor_block != 'obstacle':
                        heapq.heappush(heap, (cost + neighbor_cost, neighbor, path + [position]))

        if path_num == 0:
            print("No path found to the end position!")

    def print_path_details(self, path_num, path, cost):
        print(f"Path {path_num}:")
        for position in path:
            y, x = position
            block_type = self.grid.grid[y][x].block_type
            if block_type == 'animal':
                print(f"(A,{y},{x})", end=" ")
            elif block_type == 'free':
                print(f"(F,{y},{x})", end=" ")
        print()
        print(f"Total cost: {cost}")
        print()
    def calculate_cost(self, block_type):
        if block_type == 'free':
            return random.randint(-3, -1)
        elif block_type == 'animal':
            # Probability of getting killed by the animal
            if random.random() < 0.8:
                # Killed
                return random.randint(-4, -2)
            else:
                # Survived
                return random.randint(-3, -1)
        else:
            return 0

# Grid Class

## Description
The `Grid` class represents the game environment grid in the treasure hunt adventure. It contains methods to generate random blocks, check for paths from the start to the end position, and display the grid with player positions and paths.

## Attributes
- `width`: The width of the grid.
- `height`: The height of the grid.
- `grid`: A 2D list representing the grid with different block types.
- `start_pos`: The position of the starting point (bottom left block).
- `end_pos`: The position of the treasure (end goal, top right block).

## Methods
- `__init__(self, width, height)`: Initializes a Grid object with the specified width and height.
- `generate_random_blocks(self)`: Generates random blocks on the grid, including obstacles, animals, and free blocks.
- `has_path(self)`: Checks if there is a path from the starting point to the treasure.
- `display_grid(self, player_pos, path=None)`: Displays the grid with player positions and pat hidden treasure!


In [4]:
class Grid:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.grid = [[None for _ in range(width)] for _ in range(height)]
        self.start_pos = (height - 1, 0)  # bottom left block
        self.end_pos = (0, width - 1)     # top right block

    def generate_random_blocks(self):
        block_types = ['obstacle', 'animal', 'free']
        while True:
            self.grid = [[None for _ in range(self.width)] for _ in range(self.height)]
            self.grid[self.start_pos[0]][self.start_pos[1]] = Block('free')
            self.grid[self.end_pos[0]][self.end_pos[1]] = Block('free')

            # Generate random blocks
            for y in range(self.height):
                for x in range(self.width):
                    if self.grid[y][x] is None:
                        # 0.2 probability there is an obstacle, 0.2 probability it is an animal and 0.6 probability it is free
                        block_type = random.choices(block_types, weights=[0.2, 0.2, 0.6])[0] 
                        self.grid[y][x] = Block(block_type)

            # Check if there's a path from start to end
            if self.has_path():
                break

    def has_path(self):
        visited = set()
        stack = [self.start_pos]

        while stack:
            y, x = stack.pop()
            if (y, x) == self.end_pos:
                return True
            if (y, x) in visited:
                continue
            visited.add((y, x))

            # Add neighboring cells to the stack
            neighbors = [(y-1, x), (y+1, x), (y, x-1), (y, x+1)]
            for ny, nx in neighbors:
                if 0 <= ny < self.height and 0 <= nx < self.width and self.grid[ny][nx].block_type != 'obstacle':
                    stack.append((ny, nx))

        return False

    def display_grid(self, player_pos, path=None):
        for y in range(self.height):
            for x in range(self.width):
                if (y, x) == self.start_pos:
                    print('S', end=' ')  # starting position
                elif (y, x) == self.end_pos:
                    print('E', end=' ')  # end goal
                elif (y, x) == player_pos:
                    print('P', end=' ')  # player position
                elif path and (y, x) in path:
                    print('P', end=' ')  # path taken by the player
                else:
                    print(self.grid[y][x].block_type[0], end=' ')
            print()

# GridGUI Class

## Description
The `GridGUI` class represents the graphical user interface for visualizing the game grid and performing actions like updating the grid and showing the path taken by the player. It utilizes Tkinter for GUI components and image display.

## Attributes
- `master`: The master Tkinter window.
- `grid`: An instance of the Grid class representing the game environment.
- `path`: The path taken by the player.

## Methods
- `__init__(self, master, grid)`: Initializes a GridGUI object with the master window and grid.
- `resize_image(self, img)`: Resizes the given image to fit the grid cells.
- `generate_grid(self)`: Generates the grid with blocks and displays it on the canvas.
- `update_grid(self)`: Updates the grid with random blocks and displays the changes.
- `show_path(self)`: Shows the path taken by the player and updates the grid display accordingly.
- `draw_path(self)`: Draws the path taken by the player on the grid canvas.

In [5]:
class GridGUI:
    def __init__(self, master, grid):
        self.master = master
        self.grid = grid
        self.path = None

        self.canvas = tk.Canvas(master, width=600, height=600)
        self.canvas.pack()
        self.cost_label = tk.Label(master, text="Total Cost: ", font=('Arial', 12))
        self.cost_label.pack()

        # Load images
        self.obstacle_image = self.resize_image(Image.open("obstacle.png"))
        self.animal_image = self.resize_image(Image.open("animal.png"))

        self.update_button = tk.Button(master, text="Update Grid", command=self.update_grid)
        self.update_button.pack()

        self.show_path_button = tk.Button(master, text="Show Path", command=self.show_path)
        self.show_path_button.pack()

        # Add dropdown for selecting search algorithm
        self.algorithm_var = tk.StringVar(master)
        self.algorithm_var.set("A*")  # Default value
        self.algorithm_menu = tk.OptionMenu(master, self.algorithm_var, "A*", "UCS")
        self.algorithm_menu.pack()

        self.generate_grid()
    def resize_image(self, img):
        cell_width = 600 // self.grid.width
        cell_height = 600 // self.grid.height
        return ImageTk.PhotoImage(img.resize((cell_width, cell_height)))
    def generate_grid(self):
        self.canvas.delete("all")
    
        cell_width = 600 // self.grid.width
        cell_height = 600 // self.grid.height

        # Create a font object with the desired size
        custom_font = font.Font(size=13)

        for y in range(self.grid.height):
            for x in range(self.grid.width):
                block = self.grid.grid[y][x]
                if block.block_type == 'obstacle':
                    image = self.obstacle_image
                elif block.block_type == 'animal':
                    image = self.animal_image
                else:
                    image = None

                if image:
                    self.canvas.create_image(x*cell_width, y*cell_height, anchor=tk.NW, image=image)
                else:
                    color = 'green' if block.block_type == 'free' else 'black'
                    self.canvas.create_rectangle(x*cell_width, y*cell_height, (x+1)*cell_width, (y+1)*cell_height, fill=color)
    
                if self.algorithm_var.get() == "A*":  # Only display heuristic values for A* algorithm
                    if block.block_type != 'obstacle':
                        heuristic_value = abs(y - self.grid.end_pos[0]) + abs(x - self.grid.end_pos[1])
                        self.canvas.create_text((x+0.9)*cell_width, (y+0.9)*cell_height, text=str(heuristic_value), fill='black',font=custom_font)
    
                if (y, x) == self.grid.start_pos:
                    self.canvas.create_text((x+0.5)*cell_width, (y+0.5)*cell_height, text='S', fill='black',font=custom_font)
                elif (y, x) == self.grid.end_pos:
                    self.canvas.create_text((x+0.5)*cell_width, (y+0.5)*cell_height, text='E', fill='black',font=custom_font)


    def update_grid(self):
        self.grid.generate_random_blocks()
        self.cost_label.config(text=f"Total Cost: {0}")  # Update the cost label text
        self.path = None
        self.generate_grid()

    def show_path(self):
        self.path = None
        player = Player(self.grid, self.grid.start_pos, self.grid.end_pos, self.algorithm_var.get())
        player.move_to_end()
        path = player.path_taken
        if path:
            self.path = path
            self.generate_grid()
            self.draw_path()
            total_cost = player.total_cost  # Get the total cost from the player object
            self.cost_label.config(text=f"Total Cost: {total_cost}")  # Update the cost label text

    def draw_path(self):
        cell_width = 600 // self.grid.width
        cell_height = 600 // self.grid.height

        for i in range(len(self.path) - 1):
            y1, x1 = self.path[i]
            y2, x2 = self.path[i+1]
            self.canvas.create_line((x1+0.5)*cell_width, (y1+0.5)*cell_height, (x2+0.5)*cell_width, (y2+0.5)*cell_height, fill='blue', width=3)

## Description
This section of code initializes the main window for the "Treasure Hunt In The Enchanted Forest" game. It sets up the game grid, generates random blocks, and creates the graphical user interface for interacting with the game.

## Execution Steps
1. Create a Tkinter window.
2. Set the title of the window to "Treasure Hunt In The Enchanted Forest".
3. Define the dimensions of the game grid (8x8).
4. Generate random blocks for the grid.
5. Create an instance of the GridGUI class with the root window and the generated grid.
6. Start the Tkinter event loop to run the application.

Explore the enchanted forest, find the treasure, and enjoy the adventure!


In [12]:
root = tk.Tk()
root.title("Treasure Hunt In The Enchanted Forest")

width = 10
height = 10
grid = Grid(width, height)
grid.generate_random_blocks()

grid_gui = GridGUI(root, grid)

root.mainloop()

<center><h1>Goodbye! 😊<h1></center>
